In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
merged = pd.read_csv('merged.csv', sep=',', header=0)

In [13]:
merged.columns

Index(['Date', 'MaximumTemp', 'MinimumTemp', 'AverageTemp', 'Departure', 'HDD',
       'CDD', 'Precipitation', 'NewSnow', 'DepartureTemp', 'SnowDepth',
       'Ozone', 'CO', 'Electricity(GWh)/Non-Residential',
       'Electricity(GWh)/Residential', 'Gas(Therms)/Non-Residential',
       'Gas(Therms)/Residential', 'Median_Income(dollars)', 'Population',
       'Wind', 'Solar', 'Small hydro', 'Geothermal', 'Biomass', 'Large hydro',
       'Coal', 'Nuclear', 'Natural gas', 'Unspecified'],
      dtype='object')

Convert Date column to unix

In [14]:
import datetime as dt

merged.Date = pd.to_datetime(merged.Date)
merged.Date  = (merged.Date - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
# merged.Date = merged.Date.astype(int)

In [31]:
merged = merged.replace('M', None)
merged = merged.replace('T', None)

In [38]:
merged = merged.fillna(0)

In [39]:
merged.iloc[0]

Date                                1388534400
MaximumTemp                                 65
MinimumTemp                                 35
AverageTemp                               50.0
Departure                                  3.7
HDD                                         15
CDD                                          0
Precipitation                              0.0
NewSnow                                    0.0
DepartureTemp                                0
SnowDepth                                    0
Ozone                                    0.026
CO                                           1
Electricity(GWh)/Non-Residential       6258.54
Electricity(GWh)/Residential           4736.13
Gas(Therms)/Non-Residential             100.81
Gas(Therms)/Residential                172.911
Median_Income(dollars)                   62203
Population                              485199
Wind                                     0.081
Solar                                    0.042
Small hydro  

### Run 2 regression models
1) Ozone

2) CO

In [40]:
ozone_y = merged['Ozone']
co_y = merged['CO']

In [41]:
features = merged.drop(['Ozone', 'CO'], axis=1)

Check that it was actually dropped

In [42]:
features.columns

Index(['Date', 'MaximumTemp', 'MinimumTemp', 'AverageTemp', 'Departure', 'HDD',
       'CDD', 'Precipitation', 'NewSnow', 'DepartureTemp', 'SnowDepth',
       'Electricity(GWh)/Non-Residential', 'Electricity(GWh)/Residential',
       'Gas(Therms)/Non-Residential', 'Gas(Therms)/Residential',
       'Median_Income(dollars)', 'Population', 'Wind', 'Solar', 'Small hydro',
       'Geothermal', 'Biomass', 'Large hydro', 'Coal', 'Nuclear',
       'Natural gas', 'Unspecified'],
      dtype='object')

## Regression - Ozone

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X_train_ozone, X_test_ozone, ozone_train, ozone_test = train_test_split(features, ozone_y, test_size=0.4, random_state=0)



In [59]:
from sklearn import tree

clf_ozone = tree.DecisionTreeRegressor()
clf_ozone = clf_ozone.fit(X_train_ozone, ozone_train)

In [60]:
pred_ozone = clf_ozone.predict(X_test_ozone)

In [61]:
r2_score(ozone_test, pred_ozone)

0.428712558236503

## Regression - CO

In [62]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X_train_co, X_test_co, co_train, co_test = train_test_split(features, co_y, test_size=0.4, random_state=0)

In [63]:
from sklearn import tree

clf_ozone = tree.DecisionTreeRegressor()
clf_ozone = clf_ozone.fit(X_train_co, co_train)

In [64]:
from sklearn import tree

clf_ozone = tree.DecisionTreeRegressor()
clf_ozone = clf_ozone.fit(X_train_co, co_train)

In [65]:
pred_co= clf_ozone.predict(X_test_co)

In [66]:
r2_score(co_test, pred_co)

0.5197245456226487